<a href="https://colab.research.google.com/github/melida23/CareerChatbot/blob/main/CareerChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import gradio as gr
import random

In [36]:
import gradio as gr
import random

# --- Keyword & Career Mapping ---
keyword_to_category = {
    "coding": "technology", "programming": "technology", "computers": "technology",
    "software": "technology", "math": "technology", "ai": "technology",

    "biology": "science", "chemistry": "science", "physics": "science",
    "environment": "science", "lab": "science",

    "health": "health", "nurse": "health", "medical": "health", "nutrition": "health",

    "art": "art", "drawing": "art", "music": "art", "design": "art", "fashion": "art",

    "business": "business", "management": "business", "finance": "business",
    "entrepreneur": "business", "marketing": "business",

    "teach": "education", "school": "education", "history": "education",

    "law": "law", "legal": "law", "justice": "law",

    "plumber": "skilled", "electrician": "skilled", "welding": "skilled", "tools": "skilled",
}

career_map = {
    "technology": ["Software Developer", "Cybersecurity Analyst", "UI/UX Designer", "Cloud Engineer"],
    "science": ["Biologist", "Environmental Scientist", "Physicist", "Bioinformatician"],
    "health": ["Nurse", "Public Health Educator", "Medical Lab Scientist", "Nutritionist"],
    "art": ["Graphic Designer", "Music Producer", "Art Director", "Fashion Stylist"],
    "business": ["Business Analyst", "Project Manager", "E-commerce Specialist", "HR Manager"],
    "education": ["Teacher", "Career Counselor", "Educational Technologist", "Historian"],
    "law": ["Lawyer", "Legal Advisor", "Compliance Officer"],
    "skilled": ["Electrician", "Plumber", "Welder", "Carpenter"]
}

fallback_careers = [
    # --- Life Orientation ---
    "Career Counselor", "Human Resources Officer", "Social Worker",
    "Life Coach", "Psychologist (Counseling)", "Community Development Worker",

    # --- History ---
    "Historian", "Museum Curator", "Archivist",
    "Tour Guide / Cultural Heritage Officer", "Archaeologist", "History Teacher / Lecturer",

    # --- Life Sciences ---
    "Biotechnologist", "Environmental Scientist", "Microbiologist",
    "Wildlife Conservationist", "Marine Biologist", "Medical Laboratory Technician",
    "Public Health Specialist",

    # --- Accounting ---
    "Chartered Accountant", "Auditor", "Tax Consultant",
    "Bookkeeper", "Financial Analyst", "Payroll Administrator",

    # --- Economics ---
    "Economist", "Business Analyst", "Investment Analyst",
    "Financial Planner", "Policy Advisor", "Market Research Analyst",

    # --- Business Studies ---
    "Entrepreneur / Startup Owner", "Project Manager", "Marketing Specialist",
    "Business Consultant", "Operations Manager", "Sales Executive",

    # --- Physical Sciences ---
    "Chemical Engineer", "Laboratory Technician", "Civil Engineer",
    "Renewable Energy Technician", "Forensic Scientist", "Industrial Engineer",

    # --- Mathematics ---
    "Data Scientist", "Statistician", "Software Developer",
    "Actuary", "Mathematician (Research or Education)", "Operations Research Analyst",

    # --- Mathematics Literacy ---
    "Retail Manager", "Logistics Coordinator", "Bank Teller / Credit Officer",
    "Insurance Agent", "Inventory / Supply Chain Manager", "Administrative Clerk",

    # --- Geography ---
    "Urban Planner", "GIS Specialist (Geographic Information Systems)",
    "Environmental Consultant", "Hydrologist", "Meteorologist", "Transport and Logistics Manager",

    # --- Agriculture ---
    "Agricultural Scientist", "Farm Manager", "Soil Scientist",
    "Agricultural Economist", "Food Technologist", "Animal Scientist"
]


# --- Suggest Careers Function ---
def suggest_careers(name, interests, skills, subject):
    combined_input = f"{interests} {skills} {subject}".lower()
    matched_categories = set()

    for keyword, category in keyword_to_category.items():
        if keyword in combined_input:
            matched_categories.add(category)

    suggested = set()
    for category in matched_categories:
        suggested.update(career_map.get(category, []))

    if not suggested:
        suggested.update(random.sample(fallback_careers, k=6))

    suggestions = "\n".join(f"🎓 {career}" for career in sorted(suggested))

    return f"""
<b>Hi {name}!</b> Based on your input:<br>
✅ <b>Interests:</b> {interests}<br>
✅ <b>Skills:</b> {skills}<br>
✅ <b>Favorite Subject:</b> {subject}<br><br>

<b>Here are some exciting career suggestions for you:</b><br>
{suggestions}<br><br>

💡 <i>Type 'restart' to explore again.</i>
"""

# --- Conversation State ---
conversation_state = {"step": 0, "name": "", "interests": "", "skills": "", "subject": ""}

# --- Chat Function ---
def chat_mode(user_message, history):
    step = conversation_state["step"]

    # Restart
    if user_message.strip().lower() == "restart":
        conversation_state.update({"step": 0, "name": "", "interests": "", "skills": "", "subject": ""})
        return "🔄 Restarted! Hi, I am <b>Career Path Expert</b> 🌟 What's your name?"

    # Step 0 → Greet
    if step == 0:
        conversation_state["step"] = 1
        return "👋 Hi, I am <b>Career Path Expert</b> 🌟 I help you discover your dream career!<br>What's your name?"

    # Step 1 → Capture Name
    elif step == 1:
        conversation_state["name"] = user_message
        conversation_state["step"] = 2
        return f"Hello <b>{user_message}</b>! 💡 What are your <b>interests</b>?"

    # Step 2 → Capture Interests
    elif step == 2:
        conversation_state["interests"] = user_message
        conversation_state["step"] = 3
        return "Awesome! 🎨 What are your <b>skills</b>?"

    # Step 3 → Capture Skills
    elif step == 3:
        conversation_state["skills"] = user_message
        conversation_state["step"] = 4
        return "Great! 📚 What's your <b>favorite school subject</b>?"

    # Step 4 → Suggest Careers
    elif step == 4:
        conversation_state["subject"] = user_message
        conversation_state["step"] = 5
        return suggest_careers(
            conversation_state["name"],
            conversation_state["interests"],
            conversation_state["skills"],
            conversation_state["subject"]
        )

# --- Fancy Gradio UI with Grey BG + Blue/Green Chat ---
with gr.Blocks(css="""
body {
    background-color: #e0e0e0; /* soft grey */
    font-family: 'Segoe UI', sans-serif;
}
.gradio-container {
    max-width: 750px !important;
    margin: auto;
    border-radius: 15px;
    overflow: hidden;
}
h1 {
    text-align: center;
    color: #1565c0;
}
.gr-chatbot .message.user {
    background-color: #4caf50 !important; /* green user */
    color: white !important;
    border-radius: 15px;
    padding: 10px;
}
.gr-chatbot .message.bot {
    background-color: #1e88e5 !important; /* blue bot */
    color: white !important;
    border-radius: 15px;
    padding: 10px;
}
""", title="🎯 Career Path Chatbot") as demo:

    gr.HTML("""
    <div style="text-align:center; padding: 20px;">
        <h1>💼 Career Path Chatbot</h1>
        <p style="font-size:18px; color:#1565c0;">
            Answer a few questions and discover your future career path 🚀
        </p>
    </div>
    """)

    gr.ChatInterface(fn=chat_mode,
                     chatbot=gr.Chatbot(label="Career Assistant", bubble_full_width=False))

demo.launch()

/tmp/ipython-input-613123378.py:201: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(label="Career Assistant", bubble_full_width=False))
/tmp/ipython-input-613123378.py:201: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot=gr.Chatbot(label="Career Assistant", bubble_full_width=False))
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5fae61815b6d81449d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
